In [3]:
import tensorflow as tf
tf.__version__

'1.12.0'

# Import tensorFlow

To get started, import the **`tensorflow`** module and enable eager execution. Eager execution enables a more interactive frontend to TensorFlow, the details of which we will discuss much later.

In [4]:
import tensorflow as tf

tf.enable_eager_execution()

# Tensors

A Tensor is a multi-dimensional array. <br/><br/>Similar to NumPy **`ndarray`** objects, **`Tensor`** objects have a data type and a shape. <br/><br/>Additionally, Tensors can reside in accelerator (like GPU) memory. <br/><br/>TensorFlow offers a rich library of operations (**`tf.add`**, **`tf.matmul`**, **`tf.linalg.inv`** etc.) that consume and produce Tensors. These operations automatically convert native Python types. For example:

In [6]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.encode_base64("hello world"))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG8gd29ybGQ', shape=(), dtype=string)
tf.Tensor(13, shape=(), dtype=int32)


Each Tensor has a shape and a datatype

In [7]:
x = tf.matmul([[1]], [[2, 3]])
print(x.shape)
print(x.dtype)

(1, 2)
<dtype: 'int32'>


The most obvious differences between NumPy arrays and TensorFlow Tensors are:
<font color=red>
* Tensors can be backed by accelerator memory (like GPU, TPU).
* Tensors are immutable.
</font>

# NumPy Compatibility

Conversion between **`TensorFlow Tensors`** and **`NumPy ndarrays`** is quite simple as:

* **`TensorFlow`** operations automatically convert **`NumPy ndarrays`** to Tensors.
* **`NumPy`** operations automatically convert **`Tensors`** to **`NumPy ndarrays**`.

<br/><font color=blue>Tensors can be explicitly converted to NumPy ndarrays by invoking the **`.numpy()`** method on them.</font> <br/><br/>These conversions are typically cheap as the array and Tensor share the underlying memory representation if possible. <br/><br/><font color=red>However, sharing the underlying representation isn't always possible since the Tensor may be hosted in GPU memory while NumPy arrays are always backed by host memory, and the conversion will thus involve a copy from GPU to host memory.</font>



In [9]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("\nAnd NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("\nThe .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)

And NumPy operations convert Tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]

The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


# GPU acceleration

Many TensorFlow operations can be accelerated by using the GPU for computation. <br/><br/><font color=blue>Without any annotations, TensorFlow automatically decides whether to use the GPU or CPU for an operation (and copies the tensor between CPU and GPU memory if necessary).</font> <br/><br/>Tensors produced by an operation are typically backed by the memory of the device on which the operation executed. For example:

In [10]:
x = tf.random_uniform([3, 3])

print("Is there a GPU available: "),
print(tf.test.is_gpu_available())

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

Is there a GPU available: 
True
Is the Tensor on GPU #0:  
True


### Device Names

<font color=blue>The **`Tensor.device`** property provides a fully qualified string name of the device hosting the contents of the tensor.</font> <br/><br/>This name encodes many details, such as an identifier of the network address of the host on which this program is executing and the device within that host. <br/><br/>This is required for distributed execution of a TensorFlow program. The string ends with **`GPU:<N>`** if the tensor is placed on the **`N-th`** GPU on the host.

### Explicit device Placement

The term "**placement**" in TensorFlow refers to how individual operations are assigned (placed on) a device for execution. <br/><br/>As mentioned above, when there is no explicit guidance provided, <font color=blue>TensorFlow automatically decides which device to execute an operation, and copies Tensors to that device if needed</font>. However, TensorFlow operations can be explicitly placed on specific devices using the **`tf.device`** context manager. For example:

In [13]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x, x)

  result = time.time()-start
    
  print("10 loops: {:0.2f}ms".format(1000*result))


# Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random_uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# Force execution on GPU #0 if available
if tf.test.is_gpu_available():
  print("\nOn GPU:")
  with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
    x = tf.random_uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

On CPU:
10 loops: 135.36ms

On GPU:
10 loops: 0.55ms


# Datasets

This section demonstrates the use of the **`tf.data.Dataset`** API to build pipelines to feed data to your model. It covers:
<font color=blue>
* Creating a **`Dataset`**.
* Iteration over a **`Dataset`** with eager execution enabled.
</font>

We recommend using the **`Datasets`** API for building performant, complex input pipelines from simple, re-usable pieces that will feed your model's training or evaluation loops.

If you're familiar with TensorFlow graphs, <font color=blue>the API for constructing the **`Dataset`** object remains exactly the same when eager execution is enabled, but the process of iterating over elements of the dataset is slightly simpler.</font> <br/><br/>You can use Python iteration over the [**`tf.data.Dataset`**](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) object and do not need to explicitly create an [**`tf.data.Iterator`**](https://www.tensorflow.org/api_docs/python/tf/data/Iterator) object. As a result, the discussion on iterators in the [TensorFlow Guide](https://www.tensorflow.org/guide/datasets) is not relevant when eager execution is enabled.

### Create a source Dataset

Create a __source__ dataset using one of the factory functions like [**`Dataset.from_tensors`**](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensors), [**`Dataset.from_tensor_slices`**](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices) or using objects that read from files like [**`TextLineDataset`**](https://www.tensorflow.org/api_docs/python/tf/data/TextLineDataset) or [**`TFRecordDataset`**](https://www.tensorflow.org/api_docs/python/tf/data/TFRecordDataset). <br/><br/>See the [TensorFlow Guide](https://www.tensorflow.org/guide/datasets#reading_input_data) for more information.

In [21]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

### Apply transformations

Use the transformations functions like [**`map`**](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map), [**`batch`**](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch), [**`shuffle`**](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle) etc. <font color=blue>to apply transformations to the records of the dataset.</font> <br/><br/>See the [API documentation](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) for [**`tf.data.Dataset`**](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) for details.

In [22]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)
print(ds_tensors)
ds_file = ds_file.batch(2)
print(ds_file)

<BatchDataset shapes: (?,), types: tf.int32>
<BatchDataset shapes: (?,), types: tf.string>


### Iterate

When eager execution is enabled **`Dataset`** objects support iteration. If you're familiar with the use of **`Datasets`** in TensorFlow graphs, <font color=red>note that there is no need for calls to **`Dataset.make_one_shot_iterator()`** or **`get_next()`** calls.</font>

In [23]:
print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

print('\nElements in ds_file:')
for x in ds_file:
  print(x)

Elements of ds_tensors:
tf.Tensor([4 1], shape=(2,), dtype=int32)
tf.Tensor([16 25], shape=(2,), dtype=int32)
tf.Tensor([ 9 36], shape=(2,), dtype=int32)

Elements in ds_file:
tf.Tensor([b'Line 1' b'Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'Line 3' b'  '], shape=(2,), dtype=string)
